Dokument ma na celu wytłumaczenie na przykładzie w jaki sposób korzystać z funkcji ***add_city*** służącej do detekcji miast.

# Opis funkcji

Funkcja ***add_city*** zwraca sparkowy DataFrame, w którym zostają dodane kolumny według przyznanych flag w parametrach funkcji.
Jednym z parametrów funkcji jest *path*, jest to ścieżka do pliku .csv z lokalizacjami. Domyślnie ustawiona na plik w wspólnym katalogu projektu. \
Parametry:  \
***add_city_flag*** - Dodanie kolumny *City_flag* do DataFramu z wartościami logicznymi określającymi, czy dana lokalizacja odpowiada miastu, zgodnie z wczytanym plikiem. \
***add_city_name*** - Dodanie kolumny *City* do DataFramu z nazwą miasta. \
***add_state*** - Dodanie kolumny *State* do DataFramu z nazwą stanu. \
***add_country*** - Dodanie kolumny *Country* do DataFramu z nazwą kraju. \
Jeżeli brak informacji w wczytanym pliku z lokalizacjami, to wstawiana jest wartość NULL.

# Przykład użycia

Przed użyciem funkcji konieczne jest utworzenie środowiska w sparku oraz połączenie z dyskiem google.

## 1. Wczytanie danych w sparku

Utworzenie środowiska pyspark do obliczeń:

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

import findspark
findspark.init()

In [2]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType, StringType, StructType

from google.colab import drive

Utowrzenie sesji:

In [3]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Połączenie z dyskiem:

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Wczytanie danych NASA znajdujących się na dysku w sparku:

In [5]:
# Wczytanie zbioru sampled w celu pobrania nazw kolumn
sampled = pd.read_csv('/content/drive/MyDrive/BigMess/NASA/sampled_NASA_200k.csv')

# Utworzenie schematu określającego typ zmiennych
schemat = StructType()
for i in sampled.columns:
  if i == "Date":
    schemat = schemat.add(i, StringType(), True)
  else:
    schemat = schemat.add(i, FloatType(), True)

In [6]:
nasa = spark.read.format('csv').option("header", True).schema(schemat).load('/content/drive/MyDrive/BigMess/NASA/NASA.csv')
nasa.show(5)

+---------+-------+------+---------+---------+---------+---------+---------+---------+----------+---+-----+-----+---------+---+----------+---+---------+---------+---+---------+--------+------------+-------------+--------------+---------------+------------+-------------+--------------+---------------+-------------+-------------+----------+------------+-------------+--------------+---------------+---------------+---------------+---------+----------+---------+----------+-------+------------+-----------+------------+----------+----------+----------+----------+---------+------------+---------+----------+----------+
|      lon|    lat|  Date|   SWdown|   LWdown|    SWnet|    LWnet|      Qle|       Qh|        Qg| Qf|Snowf|Rainf|     Evap| Qs|       Qsb|Qsm| AvgSurfT|   Albedo|SWE|SnowDepth|SnowFrac|SoilT_0_10cm|SoilT_10_40cm|SoilT_40_100cm|SoilT_100_200cm|SoilM_0_10cm|SoilM_10_40cm|SoilM_40_100cm|SoilM_100_200cm|SoilM_0_100cm|SoilM_0_200cm| RootMoist|SMLiq_0_10cm|SMLiq_10_40cm|SMLiq_40_100cm|

## 2. Użycie funkcji

W tym przykładzie funkcja zostanie zaimportowana z katalogu na dysku. Jednak po dodaniu pliku do głównej gałęzi na GitHubie będzie można ją pobierać
stamtąd bezpośrednio.

In [7]:
import sys
sys.path.insert(0,"/content/drive/MyDrive/Colab Notebooks/Analiza BIG DATA/Sprint 1")

In [8]:
import city_detector

Dodamy wszystkie możliwe kolumny.

In [9]:
nasa=city_detector.add_city(nasa, add_city_name=True,add_state=True, add_country=True)

In [10]:
nasa

DataFrame[lon: float, lat: float, Date: string, SWdown: float, LWdown: float, SWnet: float, LWnet: float, Qle: float, Qh: float, Qg: float, Qf: float, Snowf: float, Rainf: float, Evap: float, Qs: float, Qsb: float, Qsm: float, AvgSurfT: float, Albedo: float, SWE: float, SnowDepth: float, SnowFrac: float, SoilT_0_10cm: float, SoilT_10_40cm: float, SoilT_40_100cm: float, SoilT_100_200cm: float, SoilM_0_10cm: float, SoilM_10_40cm: float, SoilM_40_100cm: float, SoilM_100_200cm: float, SoilM_0_100cm: float, SoilM_0_200cm: float, RootMoist: float, SMLiq_0_10cm: float, SMLiq_10_40cm: float, SMLiq_40_100cm: float, SMLiq_100_200cm: float, SMAvail_0_100cm: float, SMAvail_0_200cm: float, PotEvap: float, ECanop: float, TVeg: float, ESoil: float, SubSnow: float, CanopInt: float, ACond: float, CCond: float, RCS: float, RCT: float, RCQ: float, RCSOL: float, RSmin: float, RSMacr: float, LAI: float, GVEG: float, Streamflow: float, City_flag: boolean, City: string, State: string, Country: string]

W obiekcie nasa widzimy dodanie nowych kolumn.

In [11]:
nasa.show(4)

+---------+-------+------+---------+---------+---------+---------+---------+---------+----------+---+-----+-----+---------+---+---+---+---------+---------+---+---------+--------+------------+-------------+--------------+---------------+------------+-------------+--------------+---------------+-------------+-------------+----------+------------+-------------+--------------+---------------+---------------+---------------+---------+----------+---------+----------+-------+------------+-----------+------------+----------+----------+----------+----------+-----+------------+---------+----------+----------+---------+-------------------+-------------------+-------+
|      lon|    lat|  Date|   SWdown|   LWdown|    SWnet|    LWnet|      Qle|       Qh|        Qg| Qf|Snowf|Rainf|     Evap| Qs|Qsb|Qsm| AvgSurfT|   Albedo|SWE|SnowDepth|SnowFrac|SoilT_0_10cm|SoilT_10_40cm|SoilT_40_100cm|SoilT_100_200cm|SoilM_0_10cm|SoilM_10_40cm|SoilM_40_100cm|SoilM_100_200cm|SoilM_0_100cm|SoilM_0_200cm| RootMoist|SM

Przykładowe wyniki.

In [12]:
nasa.select('lon','lat','City_flag','City','City_flag','State','Country').limit(10).toPandas()

,lon,lat,City_flag,City,City_flag,State,Country
0,-97.9375,25.5625,False,None,False,Tamaulipas,México
1,-108.6875,25.6875,False,None,False,Sinaloa,México
2,-99.4375,25.6875,False,None,False,Nuevo León,México
3,-98.3125,25.9375,False,None,False,Tamaulipas,México
4,-98.0625,25.9375,False,None,False,Tamaulipas,México
5,-100.0625,26.1875,False,None,False,Nuevo León,México
6,-100.6875,26.4375,False,None,False,Nuevo León,México
7,-100.1875,26.6875,False,None,False,Nuevo León,México
8,-109.0625,26.8125,False,None,False,Sonora,México
9,-99.4375,27.1875,False,None,False,Tamaulipas,México


In [13]:
nasa.select('lon','lat','City_flag','City','City_flag','State','Country').filter(nasa.City_flag==True).limit(10).toPandas()

,lon,lat,City_flag,City,City_flag,State,Country
0,-87.5625,33.1875,True,Tuscaloosa,True,Alabama,United States
1,-112.1875,33.6875,True,Glendale,True,Arizona,United States
2,-78.5625,35.8125,True,Raleigh,True,North Carolina,United States
3,-87.6875,37.1875,True,Dawson Springs,True,Kentucky,United States
4,-85.8125,38.0625,True,Louisville,True,Kentucky,United States
5,-78.9375,40.0625,True,Stonycreek Township,True,Pennsylvania,United States
6,-75.3125,40.0625,True,Lower Merion Township,True,Pennsylvania,United States
7,-77.9375,40.4375,True,Henderson Township,True,Pennsylvania,United States
8,-78.4375,40.8125,True,Bigler Township,True,Pennsylvania,United States
9,-78.8125,41.1875,True,Washington Township,True,Pennsylvania,United States
